In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/signapoop/Desktop/fyp-graph-clustering'

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import pickle 
import numpy as np
import pandas as pd
import torch
import os
from timeit import default_timer as timer

from sklearn.manifold import TSNE
from util.evaluation_metrics import trustworthiness, nearest_neighbours_generalisation_accuracy

In [3]:
from core.SimpleNet import SimpleNet
from core.OldGraphConvNet2 import OldGraphConvNet2
from core.DataEmbeddingGraph import DataEmbeddingGraph
from core.EmbeddingDataSet import EmbeddingDataSet
from util.plot_embedding import plot_embedding, plot_embedding_subplot
from util.evaluation_metrics import evaluate_net_metrics, evaluate_embedding_metrics
from core.DimReduction import DimReduction

In [4]:
if torch.cuda.is_available():
    print('cuda available')
    device = 'cuda'
else:
    print('cuda not available')
    device = 'cpu'

cuda not available


In [5]:
dataset_name = 'cora'
parent_dir = os.path.abspath('..')
input_dir = os.path.join(parent_dir, 'data')
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)
dataset.create_all_data(split_batches=False, shuffle=True)
dataset.summarise()

Name of dataset = cora
Input dimension = 1433
Number of training samples = None
Training labels = True
Graph information = True


In [6]:
net_parameters = {}
net_parameters['n_components'] = 2
net_parameters['D'] = dataset.input_dim # input dimension
net_parameters['H'] = 50 # number of hidden units
net_parameters['L'] = 10 # number of hidden layers

In [68]:
# Graph net
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)
dataset.create_all_data(split_batches=False, shuffle=True)

net_1 = OldGraphConvNet2(net_parameters)
if torch.cuda.is_available(): net_1.cuda()
root = 'results/cora_29/'
filename = root + 'graph_net_800.pkl'
checkpoint = torch.load(filename, map_location=device)
net_1.load_state_dict(checkpoint['state_dict'])
net_1.eval()

trust_score, one_nn_score, five_nn_score, time_elapsed = evaluate_net_metrics(dataset.all_data, net_1, 'cosine')
print("Trustworthy score = {:.4f}".format(trust_score))
print("1-NN generalisation accuracy = {:.4f}".format(one_nn_score))
print("5-NN generalisation accuracy = {:.4f}".format(five_nn_score))
print("Average time to compute (s) = {:.2f}".format(time_elapsed))

Trustworthy score = 0.5348
1-NN generalisation accuracy = 0.2773
5-NN generalisation accuracy = 0.3224
Average time to compute (s) = 1.09


In [30]:
# Simple net
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)
dataset.create_all_data(split_batches=False, shuffle=True)

net_2 = SimpleNet(net_parameters)
if torch.cuda.is_available(): net_2.cuda()
root = 'results/cora_28/'
filename = root + 'simple_net_1000.pkl'
checkpoint = torch.load(filename, map_location=device)
net_2.load_state_dict(checkpoint['state_dict'])
net_2.eval()

trust_score, one_nn_score, five_nn_score, time_elapsed = evaluate_net_metrics(dataset.all_data, net_2, 'cosine')
print("Trustworthy score = {:.4f}".format(trust_score))
print("1-NN generalisation accuracy = {:.4f}".format(one_nn_score))
print("5-NN generalisation accuracy = {:.4f}".format(five_nn_score))
print("Average time to compute (s) = {:.2f}".format(time_elapsed))

Trustworthy score = 0.7599
1-NN generalisation accuracy = 0.5971
5-NN generalisation accuracy = 0.6377
Average time to compute (s) = 0.07


In [40]:
# tSNE
dataset.create_all_data(split_batches=False, shuffle=True)

embedder = TSNE(n_components=2, metric='cosine', method='exact', perplexity=30, verbose=0)

trust_score, one_nn_score, five_nn_score, time_elapsed = evaluate_embedding_metrics(dataset.all_data, embedder, 'cosine')
print("Trustworthy score = {:.4f}".format(trust_score))
print("1-NN generalisation accuracy = {:.4f}".format(one_nn))
print("5-NN generalisation accuracy = {:.4f}".format(five_nn_score))
print("Average time to compute (s) = {:.2f}".format(time_elapsed_score))

Trustworthy score = 0.8974
1-NN generalisation accuracy = 0.6911
5-NN generalisation accuracy = 0.6694
Average time to compute (s) = 136.49


In [69]:
time_start = timer()

if torch.cuda.is_available():   
    y_pred = net_1.forward(dataset.all_data[0]).cpu().detach().numpy()
else:   
    y_pred = net_1.forward(dataset.all_data[0]).detach().numpy()
    
time_elapsed = timer() - time_start

y_pred.shape

(2708, 2)

In [72]:
import scipy.sparse as sp
import bokeh.plotting as bp
from bokeh.plotting import show
from bokeh.models.glyphs import Segment
from bokeh.io import output_notebook
output_notebook()

X_emb = y_pred
labels = np.array([int(l) for l in dataset.labels])
target_names = [str(i) for i in range(7)]
named_labels = [target_names[l] for l in labels]
adj = sp.coo_matrix(dataset.adj_matrix)  # sparse matrix

# 20 colors
colormap = np.array([
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c",
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5",
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f",
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5"
])

plot_fig = bp.figure(plot_width=900, plot_height=650,
                     tools="pan, wheel_zoom, box_zoom, reset, previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

data_dict = {'x': X_emb[:, 0],
             'y': X_emb[:, 1],
             'color': colormap[labels],
             'label': named_labels}

mySource = bp.ColumnDataSource(data_dict)

classA = labels[adj.row]
classB = labels[adj.col]
mask = classA==classB
edge_colormask = mask * (classA + 1) - 1

edge_colormask = edge_colormask[mask]
p0 = X_emb[adj.row[mask],:]
p1 = X_emb[adj.col[mask],:]

source = bp.ColumnDataSource(dict(
        x0=p0[:,0],
        y0=p0[:,1],
        x1=p1[:,0],
        y1=p1[:,1],
        color=colormap[edge_colormask]
    )
)

glyph = Segment(x0="x0", y0="y0", x1="x1", y1="y1", line_color="color", line_width=1, line_alpha=0.1)
plot_fig.add_glyph(source, glyph)

plot_fig.circle(x='x', y='y', color='color', legend='label', source=mySource)
plot_fig.legend.location = (0, 70)
new_legend = plot_fig.legend[0]
plot_fig.legend[0].plot = None
plot_fig.add_layout(new_legend, 'right')
plot_fig.legend.label_text_font_size = '7pt'

show(plot_fig)


Loading BokehJS ...

## Test on full dataset

In [68]:
fname = 'cora_full.pkl'
with open(input_dir + '/cora/' + fname, 'rb') as f:
    [X, labels, X_emb, W] = pickle.load(f)

In [77]:
all_indices = np.arange(0, X.shape[0])
test_indices = np.arange(X.shape[0]-542, X.shape[0])

In [80]:
np.random.shuffle(all_indices)

In [90]:
new_test_indices = np.array([i for i,idx in enumerate(all_indices) if idx in test_indices])
len(new_test_indices)

542

In [81]:
X_full = X[all_indices]
labels_full = labels[all_indices]
X_emb_full = X_emb[all_indices]
W_full = W[:, all_indices][all_indices, :]

In [83]:
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)
dataset.inputs = X_full
dataset.labels = labels_full
dataset.X_emb = X_emb_full
dataset.adj_matrix = W_full

In [84]:
# Split dataset into batches
dataset.create_all_data(split_batches=True, shuffle=False)
print(sum([len(G.labels) for G in dataset.all_data]))

2708


In [85]:
# Predict batch by batch
time_start = timer()

y_pred = np.zeros((2708,2))
i = 0
for G in dataset.all_data:
    if torch.cuda.is_available():   
        y_pred_batch = net_1.forward(G).cpu().detach().numpy()
    else:    
        y_pred_batch = net_1.forward(G).detach().numpy()
    y_pred[i: i+len(G.labels)] = y_pred_batch
    i += len(G.labels)

time_elapsed = timer() - time_start

In [91]:
mask = new_test_indices

In [93]:
trust_score = trustworthiness(dataset.inputs[mask], y_pred[mask], n_neighbors=5, metric='cosine')
one_nn_score = nearest_neighbours_generalisation_accuracy(y_pred[mask], dataset.labels.numpy()[mask], 1)
five_nn_score = nearest_neighbours_generalisation_accuracy(y_pred[mask], dataset.labels.numpy()[mask], 5)
print("Trustworthy score = {:.4f}".format(trust_score))
print("1-NN generalisation accuracy = {:.4f}".format(one_nn_score))
print("5-NN generalisation accuracy = {:.4f}".format(five_nn_score))
print("Average time to compute (s) = {:.2f}".format(time_elapsed))

Trustworthy score = 0.6778
1-NN generalisation accuracy = 0.4556
5-NN generalisation accuracy = 0.5793
Average time to compute (s) = 0.09


In [58]:
all_indices = np.arange(0, dataset.inputs.shape[0])
np.random.shuffle(all_indices)

In [60]:
from copy import copy
foo = copy(dataset.adj_matrix)
foo = foo[all_indices,:][:,all_indices]

In [61]:
adj = foo.toarray()
adj.shape

(2708, 2708)

In [62]:
def check_symmetric(a, tol=1e-8):
    return np.allclose(a, a.T, atol=tol)

In [63]:
check_symmetric(adj)

True